In [1]:
import pandas as pd
import numpy as np
import re
import os
import datetime
import matplotlib.pyplot as plt

In [2]:
file_paths = [
    '../data/extracted_data/barcelona_business_intelligence.csv',
    '../data/extracted_data/barcelona_data_analyst.csv',
    '../data/extracted_data/barcelona_data_engineer.csv',
    '../data/extracted_data/barcelona_data_scientist.csv',
    '../data/extracted_data/madrid_business_intelligence.csv',
    '../data/extracted_data/madrid_data_analyst.csv',
    '../data/extracted_data/madrid_data_engineer.csv',
    '../data/extracted_data/madrid_data_scientist.csv'
]

# Concatenate CSV files
dfs = [pd.read_csv(file) for file in file_paths]
df = pd.concat(dfs, ignore_index=True)

# Save dataframe
df.to_csv('../data/spain_jobposts_combined.csv', index=False)

In [3]:
df = pd.read_csv('../data/spain_jobposts_combined.csv',encoding='utf-8')
df.sample(1)

,title,company_name,location,posted_time_ago,seniority_level,employment_type,job_function,industries,description
2235,PL/SQL Developer,Babel,Madrid Community of Madrid Sp...,...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting ...,Babel es un equipo internacional de más de ...


In [4]:
df.shape

(3124, 9)

## Drop duplicated rows

In [5]:
# Find duplicates based on 'title' and 'company' columns 
duplicate_mask = df.duplicated(subset=['title', 'company_name'], keep=False)

# Get both original and duplicate rows for double-check
original_and_duplicates = df[duplicate_mask].sort_values(by=['title', 'company_name'])
original_and_duplicates

,title,company_name,location,posted_time_ago,seniority_level,employment_type,job_function,industries,description
1938,Data Analyst / Engineer - PySpark y Azure,EXCELIA,Madrid Community of Madrid Sp...,...,Associate,Full-time,Engineering and Information Techn...,IT Services and IT Consulting ...,excelia es una firma multinacional de Consu...
2507,Data Analyst / Engineer - PySpark y Azure,EXCELIA,Madrid Community of Madrid Sp...,...,Associate,Full-time,Engineering and Information Techn...,IT Services and IT Consulting ...,excelia es una firma multinacional de Consu...
2566,Data Analyst / Engineer - PySpark y Azure,EXCELIA,Madrid Community of Madrid Sp...,...,Associate,Full-time,Engineering and Information Techn...,IT Services and IT Consulting ...,excelia es una firma multinacional de Consu...
1934,Ingeniero/ a de Datos SQL Big data y Azure,Tecnicas Reunidas ...,Madrid Community of Madrid Sp...,...,Mid-Senior level,Full-time,Engineering,IT Services and IT Consulting ...,Técnicas Reunidas Group (TR) es una empresa...
2288,Ingeniero/ a de Datos SQL Big data y Azure,Tecnicas Reunidas ...,Madrid Community of Madrid Sp...,...,Mid-Senior level,Full-time,Engineering,IT Services and IT Consulting ...,Técnicas Reunidas Group (TR) es una empresa...
...,...,...,...,...,...,...,...,...,...
1557,Técnico/a de desarrollo (.net y Sql),Grupo Crit,Madrid Community of Madrid Sp...,...,Entry level,Full-time,Management and Manufacturing ...,Human Resources Services,En Grupo Crit desde del departament...
2406,UI Software Engineer (C++) - Madden Core,Electronic Arts (EA) ...,Madrid Community of Madrid Sp...,...,Associate,Full-time,Engineering,IT Services and IT Consulting Soft...,EA SPORTS is one of the leading sports ente...
2575,UI Software Engineer (C++) - Madden Core,Electronic Arts (EA) ...,Madrid Community of Madrid Sp...,...,Associate,Full-time,Engineering,IT Services and IT Consulting Soft...,EA SPORTS is one of the leading sports ente...
716,Workplace Analytics Engineer,Canonical,Barcelona Catalonia Spain ...,...,Entry level,Full-time,Information Technology,Technology Information and Interne...,Bring your people analytics social...


In [6]:
# Drop the duplicate rows
df.drop_duplicates(subset=['title', 'company_name'], keep='first', inplace=True)

## Create column: "job_id"

In [7]:
df['job_id'] = range(1, len(df) + 1)

## Cleaning column: "title"

In [8]:
def clean_title(title):
    title_lower = title.lower()
    if 'machine' in title_lower:
        return 'Machine Learning Engineer'
    if 'artificial' in title_lower or 'AI' in title or 'ML' in title:
        return 'Machine Learning Engineer'
    elif 'cloud' in title_lower:
        return 'Cloud Engineer'
    elif 'software' in title_lower:
        return 'Software Engineer'
    elif 'dev' in title_lower:
        return 'Software Engineer'
    elif re.search(r'\bBI\b', title) or re.search(r'\bbusiness\s*(?!data\s*)\w*\s*analyst\b|\bbusiness\s*(?!data\s*)\w*\s*intelligence\b|\bfinancial\s*analyst\b', title_lower):
        return 'Business Analyst'
    elif 'engineer' in title_lower and 'data' in title_lower:
        return 'Data Engineer'
    elif 'software' in title_lower:
        return 'Software Engineer'
    elif 'scien' in title_lower:
        return 'Data Scientist'
    elif 'cient' in title_lower:
        return 'Data Scientist'
    elif re.search(r'\bdata\b|\banalytics?\b', title_lower):
        return 'Data Analyst'
    elif re.search(r'\bdata\b.*\b(analyst|analytics|analysis)\b|\banalytics?\b', title_lower):
        return 'Data Analyst'
    elif 'system' in title_lower:
        return 'Software Engineer'
    elif 'program' in title_lower:
        return 'Software Engineer'
    elif 'ingen' in title_lower and 'datos' in title_lower:
        return 'Data Engineer'
    elif 'analista'in title_lower and 'datos' or 'digital' in title_lower:
        return 'Data Analyst'
    elif 'desarro' in title_lower:
        return 'Software Engineer'
    elif 'backend' in title_lower:
        return 'Software Engineer'
    elif 'frontend' in title_lower:
        return 'Software Engineer'
    elif 'full' in title_lower:
        return 'Software Engineer'
    
    else:
        return None

In [9]:
# Apply the function and create the 'title_cleaned' column in clear data role categories 
df.loc[:, 'title_cleaned'] = df['title'].apply(clean_title)

In [10]:
df['title_cleaned'].value_counts()

title_cleaned
Software Engineer            455
Data Analyst                 314
Data Engineer                278
Data Scientist               172
Business Analyst             146
Machine Learning Engineer     88
Cloud Engineer                46
Name: count, dtype: int64

In [11]:
df['title_cleaned'].isnull().sum()

391

In [12]:
df.dropna(subset=['title_cleaned'], inplace=True)

## Cleaning column: "seniority_level"

In [13]:
df['seniority_level'] = df['seniority_level'].str.strip()

In [14]:
replace_dict = {
    'Mid-Senior level': 'Mid-Senior',
    'Entry level': 'Junior',
    'Associate': 'Junior',
    'Not Applicable': 'Not-Specified',
    'Internship': 'Internship',
    'Director': 'Director',
    'Executive': 'Manager',
}

df['seniority_level'] = df['seniority_level'].replace(replace_dict)

In [15]:
df['seniority_level'].value_counts()

seniority_level
Junior           628
Mid-Senior       574
Not-Specified    204
Internship        19
Director           7
Manager            5
Name: count, dtype: int64

## Create column: "experience_level"

In [16]:
def categorize_experience(title):
    title_lower = title.lower()
    if any(word in title_lower for word in [ 'intern','internship']):
        return 'Internship'
    elif any(word in title_lower for word in ['manager', 'executive', 'principal', 'staff']):
        return 'Manager'
    elif any(word in title_lower for word in ['director','associate director']):
        return 'Director'
    elif any(word in title_lower for word in ['mid-senior','mid level','mid-level','ii','specialist']):
        return 'Mid-Senior'
    elif any(word in title_lower for word in ['jr', 'jr.','junior','entry', 'associate','1']):
        return 'Junior'
    elif any(word in title_lower for word in ['sr.', 'sr', 'senior','exper','3','4','lead','steward']):
        return 'Senior'
    elif any(word in title for word in ['III','IV']):
        return 'Senior'
    elif 'II' in title:
        return 'Mid-Senior'
    elif re.search(r'\bI\b', title):
        return 'Junior'
    else:
        return None

In [17]:
# Apply the function to create the 'experience_level' column
df['experience_level'] = df['title'].apply(categorize_experience)

In [18]:
df['experience_level'].isnull().sum()

1028

In [19]:
# Fill null cells in 'experience_level' with values from 'seniority_level' where 'experience_level' is null
df['experience_level'].fillna(df['seniority_level'], inplace=True)

In [20]:
df['experience_level'].isnull().sum()

40

In [21]:
df.head(2)

,title,company_name,location,posted_time_ago,seniority_level,employment_type,job_function,industries,description,job_id,title_cleaned,experience_level
0,Business Analyst (Bangkok Based relocation pr...,Agoda,Barcelona Catalonia Spain ...,...,Junior,Full-time,Research Analyst and Informatio...,Technology Information and Interne...,About AgodaAgoda is an online travel bookin...,1,Business Analyst,Junior
1,Senior Data Analyst (Product Team) (Bangkok Ba...,Agoda,Barcelona Catalonia Spain ...,...,Junior,Full-time,Information Technology,Technology Information and Interne...,About AgodaAgoda is an online travel bookin...,2,Data Analyst,Senior


## Creating a New Column: "Role"

In [22]:
def get_role_name(row):
    title_cleaned = row['title_cleaned']
    experience_level = row['experience_level']
    
    if experience_level == "Not-Specified":
        return title_cleaned
    elif title_cleaned is None:
        return None
    else:
        return f"{title_cleaned} {experience_level}"

In [23]:
df['role'] = df.apply(get_role_name, axis=1)

## Cleaning column: "location"

In [24]:
# Convert the column to lowercase for case-insensitive matching
df.loc[:, 'location'] = df['location'].str.lower()

# Set conditions for replacing values
condition_catalonia = df['location'].str.contains('barcelona|catalonia|manresa|granollers', case=False)
condition_madrid = df['location'].str.contains('madrid', case=False)

# Replace values based on conditions using .loc to avoid SettingWithCopyWarning
df.loc[condition_catalonia, 'location'] = 'Catalonia'
df.loc[condition_madrid, 'location'] = 'Madrid'

## Creating new column: "via"

In [25]:
df['via'] = 'Linkedin'

## Cleaning column: "employment_type"

In [26]:
# Changing name to have similar columns to the US dataset
df = df.rename(columns={'employment_type': 'schedule_type'})

In [27]:
df['schedule_type'] = df['schedule_type'].str.strip()

In [28]:
df['schedule_type'] = df['schedule_type'].replace({
    'Other': 'Full-time',
    'Temporary': 'Contractor',
    'Contract': 'Contractor'})

In [29]:
df['schedule_type'].value_counts()

schedule_type
Full-time     1431
Contractor      47
Part-time       13
Internship       8
Name: count, dtype: int64

## Cleaning column: "posted_time_ago"

In [30]:
# Remove leading and trailing whitespace from 'posted_time_ago' column
df['posted_time_ago'] = df['posted_time_ago'].str.strip()

In [31]:
from datetime import datetime

# Dictionary to map time units to their corresponding pandas timedelta arguments
time_units = {
    'month': 'D',  # Change to 'D' to avoid ambiguity
    'year': 'D',  # Change to 'D' to avoid ambiguity
    'week': 'W',
    'day': 'D',
    'hour': 'h',
    'minute': 'm',
    'second': 's'
}

# Function to convert time strings to timedelta
def time_string_to_timedelta(time_str):
    # Extract numerical value and unit
    value, unit, _ = time_str.split()
    # Convert value to an integer
    value = int(value)
    # Remove 's' (plural) if present 
    unit = unit.rstrip('s')
    # Map the unit to the corresponding timedelta argument
    unit = time_units.get(unit.lower()) 
    # Create timedelta object
    return pd.to_timedelta(value, unit=unit)

In [32]:
# Apply the function to create a new column with timedelta values
df['time_difference'] = df['posted_time_ago'].apply(time_string_to_timedelta)

# Calculate the date when the job was posted (03.Dec.2023 is the reference date)
today = pd.to_datetime(datetime.now().date())
df['posted_date'] = today - df['time_difference']

print(df[['posted_time_ago', 'posted_date']])

     posted_time_ago posted_date
0        2 weeks ago  2023-11-20
1         1 week ago  2023-11-27
2         3 days ago  2023-12-01
3         1 week ago  2023-11-27
4         1 week ago  2023-11-27
...              ...         ...
3114    5 months ago  2023-11-29
3118     3 weeks ago  2023-11-13
3119    6 months ago  2023-11-28
3120      4 days ago  2023-11-30
3122     3 weeks ago  2023-11-13

[1499 rows x 2 columns]


## Creating column: "skills"

In [33]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Define the function to tokenize descriptions and remove stopwords
def tokenize_and_remove_stopwords(description):
    # Convert to lowercase and tokenize
    tokens = word_tokenize(description.lower())
    # Remove non-alphabetic tokens and stopwords
    return [token for token in tokens if token.isalpha() and token not in stop_words]

# Set of English stopwords
stop_words = set(stopwords.words('english'))

# Tokenize the descriptions and remove stopwords
df['description_tokens'] = df['description'].apply(tokenize_and_remove_stopwords)

# Define a list of technical skills relevant to data roles
skills_list = [ "tableau","excel","power_bi","sas","word","powerpoint","sap","ssis","looker","qlik","alteryx",
    "spss","ssrs","outlook","dax","sharepoint","splunk","microstrategy","cognos","visio","google sheets","spreadsheet",
    "ms access","datarobot","sql server","mysql","cassandra","postgresql","mongodb","elasticsearch","dynamodb","redis","db2","neo4j",
    "mariadb","firebase","couchbase","sqlite","firestore","couchdb","aws","azure","snowflake","databricks","redshift","gcp","oracle","bigquery","aurora",
    "vmware","ibm cloud","firebase","watson","openstack","heroku","digital ocean", "colocation","ovh","linode","spark","hadoop","kafka","airflow","pyspark","pandas","tensorflow","pytorch","numpy","scikit-learn",
    "keras","jupyter","react","matplotlib","spring","gdpr","plotly","seaborn","graphql","nltk","opencv","ggplot2",
    "selenium","mxnet","tidyverse","express","node.js","angular","flask","ruby","django","vue.js","phoenix","fastapi","jquery",
    "asp.net","ruby on rails", "react","laravel","angular.js", "asp.net core", "next.js","drupal","svelte",
    "symfony","blazor","play framework",  "gatsby","python","sql","r","java","scala","nosql","sas","go","mongodb","shell","javascript","c++","c#","c",
    "matlab","vba","bash","t-sql","powershell","html","perl","css","ruby","typescript", 'deep learning',
    'machine learning', 'nlp', 'statistics', 'data mining', 'data visualization', 'big data', 'data analysis']  # Your list of skills

# Function to extract skills from the tokenized descriptions
def extract_skills(description_tokens):
    return [skill for skill in skills_list if skill in description_tokens]


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mairagutierrez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mairagutierrez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
# Create a new column with the extracted skills for each row
df['extracted_skills'] = df['description_tokens'].apply(extract_skills)

# Display the updated dataframe with the new 'extracted_skills' column
df['extracted_skills']

0       [tableau, excel, sas, spss, python, sql, r, sa...
1            [tableau, excel, python, sql, r, statistics]
2       [tableau, excel, aws, redshift, python, sql, vba]
3                                         [microstrategy]
4                             [tableau, looker, dax, sql]
                              ...                        
3114                                          [python, c]
3118    [postgresql, redis, aws, pandas, flask, django...
3119            [pandas, pytorch, numpy, sql, statistics]
3120                            [python, sql, statistics]
3122                                                   []
Name: extracted_skills, Length: 1499, dtype: object

In [35]:
df

,title,company_name,location,posted_time_ago,seniority_level,schedule_type,job_function,industries,description,job_id,title_cleaned,experience_level,role,via,time_difference,posted_date,description_tokens,extracted_skills
0,Business Analyst (Bangkok Based relocation pr...,Agoda,Catalonia,2 weeks ago,Junior,Full-time,Research Analyst and Informatio...,Technology Information and Interne...,About AgodaAgoda is an online travel bookin...,1,Business Analyst,Junior,Business Analyst Junior,Linkedin,14 days,2023-11-20,"[agodaagoda, online, travel, booking, platform...","[tableau, excel, sas, spss, python, sql, r, sa..."
1,Senior Data Analyst (Product Team) (Bangkok Ba...,Agoda,Catalonia,1 week ago,Junior,Full-time,Information Technology,Technology Information and Interne...,About AgodaAgoda is an online travel bookin...,2,Data Analyst,Senior,Data Analyst Senior,Linkedin,7 days,2023-11-27,"[agodaagoda, online, travel, booking, platform...","[tableau, excel, python, sql, r, statistics]"
2,Junior Data Analyst,Nielsen,Catalonia,3 days ago,Not-Specified,Full-time,Information Technology,Market Research and Business Consul...,At Nielsen we believe that career ...,3,Data Analyst,Junior,Data Analyst Junior,Linkedin,3 days,2023-12-01,"[nielsen, believe, career, growth, partnership...","[tableau, excel, aws, redshift, python, sql, vba]"
3,Responsable proyectos TIC (Business Intelligence),Universidad Católica de La P...,Catalonia,1 week ago,Junior,Full-time,Project Management and Informatio...,Education Administration Programs ...,CondicionesModalidadHíbrido (remoto y prese...,4,Business Analyst,Junior,Business Analyst Junior,Linkedin,7 days,2023-11-27,"[condicionesmodalidadhíbrido, remoto, presenci...",[microstrategy]
4,Data Engineer (Power BI),Q-tech,Catalonia,1 week ago,Mid-Senior,Full-time,Engineering and Science,Technology Information and Interne...,Desde Q-Tech nos encontramos en búsqueda de...,5,Business Analyst,Mid-Senior,Business Analyst Mid-Senior,Linkedin,7 days,2023-11-27,"[desde, nos, encontramos, en, búsqueda, de, da...","[tableau, looker, dax, sql]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114,Desarrollador/a C++,Alcorce Telecomunicaciones S...,Madrid,5 months ago,Mid-Senior,Full-time,Engineering and Information Techn...,IT Services and IT Consulting ...,Nuestros equipos de desarrollo Soft...,1886,Software Engineer,Mid-Senior,Software Engineer Mid-Senior,Linkedin,5 days,2023-11-29,"[nuestros, equipos, de, desarrollo, software, ...","[python, c]"
3118,Senior Python Developer (Freelance Remote),SPACE44,Madrid,3 weeks ago,Mid-Senior,Full-time,Engineering and Information Techn...,Technology Information and Interne...,Company OverviewSPACE44 is a leading techno...,1887,Software Engineer,Senior,Software Engineer Senior,Linkedin,21 days,2023-11-13,"[company, leading, technology, company, offeri...","[postgresql, redis, aws, pandas, flask, django..."
3119,Senior Data Scientist,Fever,Madrid,6 months ago,Mid-Senior,Full-time,Engineering and Information Techn...,Technology Information and Interne...,Hi we're FeverWe're excited you ar...,1888,Data Scientist,Senior,Data Scientist Senior,Linkedin,6 days,2023-11-28,"[hi, feverwe, excited, checking, job, leading,...","[pandas, pytorch, numpy, sql, statistics]"
3120,Data Scientist (Risk),Revolut,Madrid,4 days ago,Junior,Full-time,Engineering and Information Techn...,IT Services and IT Consulting Soft...,About RevolutPeople deserve more from their...,1889,Data Scientist,Junior,Data Scientist Junior,Linkedin,4 days,2023-11-30,"[revolutpeople, deserve, money, visibility, co...","[python, sql, statistics]"


In [36]:
columns_to_drop = ['seniority_level','posted_time_ago','time_difference','description_tokens']
df.drop(columns=columns_to_drop , inplace=True)

In [37]:
# Rearrenged columns
new_order = ['job_id','title','experience_level','title_cleaned','role','company_name','location','industries','via','schedule_type','posted_date','description','extracted_skills']
df = df[new_order]

In [38]:
df.columns

Index(['job_id', 'title', 'experience_level', 'title_cleaned', 'role',
       'company_name', 'location', 'industries', 'via', 'schedule_type',
       'posted_date', 'description', 'extracted_skills'],
      dtype='object')

## Save Dataframe: Job Posts

In [39]:
df.to_csv('../data/spain_jobposts.csv', index=False) 

## Create a Dataframe based on the skills

In [40]:
# Remove empty lists [] and explode the column into separate rows
skills_df = df[df['extracted_skills'].apply(lambda x: len(x) > 0)].explode('extracted_skills')
skills_df

,job_id,title,experience_level,title_cleaned,role,company_name,location,industries,via,schedule_type,posted_date,description,extracted_skills
0,1,Business Analyst (Bangkok Based relocation pr...,Junior,Business Analyst,Business Analyst Junior,Agoda,Catalonia,Technology Information and Interne...,Linkedin,Full-time,2023-11-20,About AgodaAgoda is an online travel bookin...,tableau
0,1,Business Analyst (Bangkok Based relocation pr...,Junior,Business Analyst,Business Analyst Junior,Agoda,Catalonia,Technology Information and Interne...,Linkedin,Full-time,2023-11-20,About AgodaAgoda is an online travel bookin...,excel
0,1,Business Analyst (Bangkok Based relocation pr...,Junior,Business Analyst,Business Analyst Junior,Agoda,Catalonia,Technology Information and Interne...,Linkedin,Full-time,2023-11-20,About AgodaAgoda is an online travel bookin...,sas
0,1,Business Analyst (Bangkok Based relocation pr...,Junior,Business Analyst,Business Analyst Junior,Agoda,Catalonia,Technology Information and Interne...,Linkedin,Full-time,2023-11-20,About AgodaAgoda is an online travel bookin...,spss
0,1,Business Analyst (Bangkok Based relocation pr...,Junior,Business Analyst,Business Analyst Junior,Agoda,Catalonia,Technology Information and Interne...,Linkedin,Full-time,2023-11-20,About AgodaAgoda is an online travel bookin...,python
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3119,1888,Senior Data Scientist,Senior,Data Scientist,Data Scientist Senior,Fever,Madrid,Technology Information and Interne...,Linkedin,Full-time,2023-11-28,Hi we're FeverWe're excited you ar...,sql
3119,1888,Senior Data Scientist,Senior,Data Scientist,Data Scientist Senior,Fever,Madrid,Technology Information and Interne...,Linkedin,Full-time,2023-11-28,Hi we're FeverWe're excited you ar...,statistics
3120,1889,Data Scientist (Risk),Junior,Data Scientist,Data Scientist Junior,Revolut,Madrid,IT Services and IT Consulting Soft...,Linkedin,Full-time,2023-11-30,About RevolutPeople deserve more from their...,python
3120,1889,Data Scientist (Risk),Junior,Data Scientist,Data Scientist Junior,Revolut,Madrid,IT Services and IT Consulting Soft...,Linkedin,Full-time,2023-11-30,About RevolutPeople deserve more from their...,sql


In [41]:
# Skills categorization 

tools = ["tableau","excel","power_bi","sas","word","powerpoint","sap","ssis","looker","qlik","alteryx",
    "spss","ssrs","outlook","dax","sharepoint","splunk","microstrategy","cognos","visio","google sheets","spreadsheet",
    "ms access","datarobot"]

database = ["sql server","mysql","cassandra","postgresql","mongodb","elasticsearch","dynamodb","redis","db2","neo4j",
    "mariadb","firebase","couchbase","sqlite","firestore","couchdb"]

cloud_platforms = [ "aws","azure","snowflake","databricks","redshift","gcp","oracle","bigquery","aurora",
    "vmware","ibm cloud","firebase","watson","openstack","heroku","digital ocean", "colocation","ovh","linode"]

libraries = ["spark","hadoop","kafka","airflow","pyspark","pandas","tensorflow","pytorch","numpy","scikit-learn",
    "keras","jupyter","react","matplotlib","spring","gdpr","plotly","seaborn","graphql","nltk","opencv","ggplot2",
    "selenium","mxnet","tidyverse"]

frameworks_list = ["express","node.js","angular","flask","ruby","django","vue.js","phoenix","fastapi","jquery",
    "asp.net","ruby on rails", "react","laravel","angular.js", "asp.net core", "next.js","drupal","svelte",
    "symfony","blazor","play framework",  "gatsby"]

languages = ["python","sql","r","java","scala","nosql","sas","go","mongodb","shell","javascript","c++","c#","c",
    "matlab","vba","bash","t-sql","powershell","html","perl","css","ruby","typescript"]

In [42]:
# Function to categorize skills
def categorize_skills(skill):
    lowercase_skill = skill.lower()
    if skill in tools:
        return "Tools"
    elif skill in database:
        return "Database"
    elif skill in cloud_platforms:
        return "Cloud Platforms"
    elif skill in libraries:
        return "Libraries"
    elif skill in frameworks_list:
        return "Frameworks"
    elif skill in languages:
        return "Languages"
    else:
        return "Other"

In [43]:
# Apply the categorization function to create the 'skills_categories' column
skills_df['skills_categories'] = skills_df['extracted_skills'].apply(categorize_skills)

In [44]:
skills_df.head(2)

,job_id,title,experience_level,title_cleaned,role,company_name,location,industries,via,schedule_type,posted_date,description,extracted_skills,skills_categories
0,1,Business Analyst (Bangkok Based relocation pr...,Junior,Business Analyst,Business Analyst Junior,Agoda,Catalonia,Technology Information and Interne...,Linkedin,Full-time,2023-11-20,About AgodaAgoda is an online travel bookin...,tableau,Tools
0,1,Business Analyst (Bangkok Based relocation pr...,Junior,Business Analyst,Business Analyst Junior,Agoda,Catalonia,Technology Information and Interne...,Linkedin,Full-time,2023-11-20,About AgodaAgoda is an online travel bookin...,excel,Tools


## Save Dataframe: Skills

In [45]:
skills_df.to_csv('../data/spain_skills.csv', index=False) 